In [1]:
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")
import json

In [5]:
class question_matcher:
    def __init__(self):
        self.patterns= []
        self.q_json = self.get_questions_json()
        self.matcher = Matcher(nlp.vocab)

    def get_questions_json(self):
        with open('questions.json', 'r') as f:
            pat = json.load(f)
        return pat
    
    def define_keyword(self,question:str):
        for q in self.q_json:
            if q['keyword'] in question.lower():
                keyword = q['keyword']
                index = q['qid']
        return keyword, index

    def match_pattern(self,question):
        keyword, index = self.define_keyword(question)
        for pattern in self.q_json[index]['patterns']:
            pos = self.pattern_to_pos(pattern['pos'])
            self.matcher.add(pattern['pid'], pos,on_match=self.on_match)
            doc = nlp(question)
            matches = self.matcher(doc)
            if matches != []:
                return pattern['pattern']
            else:
                print('fail')


    def on_match(self,matcher, doc, id, matches):
        print('Matched!', matches)


    def pattern_to_pos(self, patterns:list):
        pos = []
        pattern = []
        for p in patterns:
            
            itr = []
            pattern.append(p)
            for token in p.split():
                if token != '*':
                    itr.append({'POS':token})
                else:
                    itr.append({})
            pos.append(itr)
       
        return pos
qm = question_matcher()


In [6]:
qm.match_pattern('who is barak obama')

Matched! [(1, 0, 3), (1, 0, 4)]
Matched! [(1, 0, 3), (1, 0, 4)]


'who [is,was] [NAME]'

In [7]:
qm.match_pattern('who is the president of united states')

fail
Matched! [(2, 0, 3)]


'who [is,was] [ENTITY]'

In [9]:
qm.match_pattern('give me images that contains red color')

Matched! [(1, 0, 6)]


'give me images that contain [ENTITY]'